In [2]:
import json

dataset = 'clinc150'
save_path_root = f'./results/{dataset}/100shots/'

fs_f1 = 0.0
zs_f1 = 0.0
base_icv_f1 = 0.0
diff_icv_base_f1 = 0.0
stacked_diff_icv_f1 = 0.0
for seed in [40, 41,42]:
    with open(save_path_root+f"{seed}/"+"fs_result.json", "r") as f:
        res = json.load(f)
    score = res['score']
    fs_f1+=score
    with open(save_path_root+f"{seed}/"+"zs_result.json", "r") as f:
        res = json.load(f)
    score = res['score']
    zs_f1+=score
    with open(save_path_root+f"{seed}/"+"baseline_icv_result.json", "r") as f:
        res = json.load(f)
    score = res['score']
    base_icv_f1+=score
    with open(save_path_root+f"{seed}/"+"baseline_diff_icv_result.json", "r") as f:
        res = json.load(f)
    score = res['score']
    diff_icv_base_f1+=score
    with open(save_path_root+f"{seed}/"+"stacked_diff_icv_result.json", "r") as f:
        res = json.load(f)
    score = res['score']
    stacked_diff_icv_f1+=score

print(fs_f1/3, zs_f1/3, base_icv_f1/3, diff_icv_base_f1/3, stacked_diff_icv_f1/3)


0.7424216456608482 0.3667378917378918 0.3073060247201191 0.4671201814058957 0.4804191203434069


In [3]:
import json

dataset = 'clinc150'
save_path_root = f'./results/{dataset}/100shots/'

prediction_keep_ratio = [0,0,0]
fix_ratio = [0,0,0]
change_to_wrong_ratio = [0,0,0]

for seed in [40, 41,42]:
    with open(save_path_root+f"{seed}/"+"fs_result.json", "r") as f:
        f1_res = json.load(f)
    with open(save_path_root+f"{seed}/"+"baseline_icv_result.json", "r") as f:
        bi_res = json.load(f)
    with open(save_path_root+f"{seed}/"+"baseline_diff_icv_result.json", "r") as f:
        dbi_res = json.load(f)
    with open(save_path_root+f"{seed}/"+"stacked_diff_icv_result.json", "r") as f:
        sdi_res = json.load(f)
    
    for fi, bi, dbi, sdi in zip(f1_res['result'], bi_res['result'], dbi_res['result'], sdi_res['result']):
        if fi['prediction']==bi['intervention_prediction']:
            prediction_keep_ratio[0]+=1/100
        if fi['prediction']==dbi['intervention_prediction']:
            prediction_keep_ratio[1]+=1/100
        if fi['prediction']==sdi['intervention_prediction']:
            prediction_keep_ratio[2]+=1/100
        
        if fi['prediction']!=bi['intervention_prediction'] and bi['intervention_prediction']==fi['gt']:
            fix_ratio[0]+=1/100
        if fi['prediction']!=dbi['intervention_prediction'] and dbi['intervention_prediction']==fi['gt']:
            fix_ratio[1]+=1/100
        if fi['prediction']!=sdi['intervention_prediction'] and sdi['intervention_prediction']==fi['gt']:
            fix_ratio[2]+=1/100
        
        if fi['prediction']!=bi['intervention_prediction'] and fi['prediction']==fi['gt']:
            change_to_wrong_ratio[0]+=1/100
        if fi['prediction']!=dbi['intervention_prediction'] and fi['prediction']==fi['gt']:
            change_to_wrong_ratio[1]+=1/100
        if fi['prediction']!=sdi['intervention_prediction'] and fi['prediction']==fi['gt']:
            change_to_wrong_ratio[2]+=1/100

prediction_keep_ratio = [i/3 for i in prediction_keep_ratio]
fix_ratio = [i/3 for i in fix_ratio]
change_to_wrong_ratio = [i/3 for i in change_to_wrong_ratio]

In [4]:
prediction_keep_ratio

[0.46333333333333365, 0.6433333333333339, 0.6533333333333339]

In [5]:
fix_ratio

[0.016666666666666666, 0.04333333333333333, 0.03666666666666666]

In [6]:
change_to_wrong_ratio

[0.4300000000000003, 0.2733333333333335, 0.24333333333333348]